In [12]:
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from scipy.stats import chi2_contingency

from pipelines import preprocessor, preprocessor2
from preprocess import extra_features

In [13]:
train=pd.read_csv('Datasets/Train.csv')
train=extra_features(train)
X=train.drop('cost_category', axis=1)
y= train['cost_category']

<IPython.core.display.Javascript object>

In [ ]:
col_drop=[]

for col in X.columns:
    ct=pd.crosstab(y, X[col])
    chi2, p, _, _ =chi2_contingency(ct)
    
    if p >= 0.05 and chi2 <= 3000:
        col_drop.append(col)

In [15]:
X.drop(columns=col_drop, inplace=True)

In [5]:
X=preprocessor2(X)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
len(le.classes_)

6

In [26]:
cat_features=np.where(X.dtypes != (''int', 'float''))
cat_features

<IPython.core.display.Javascript object>

(array([], dtype=int64),)

In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18506 entries, 0 to 18505
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Tour_ID                18506 non-null  object 
 1   country                18506 non-null  object 
 2   age_group              18506 non-null  object 
 3   travel_with            17431 non-null  object 
 4   total_female           18504 non-null  float64
 5   total_male             18500 non-null  float64
 6   purpose                18506 non-null  object 
 7   main_activity          18506 non-null  object 
 8   info_source            18506 non-null  object 
 9   tour_arrangement       18506 non-null  object 
 10  package_transport_int  18506 non-null  object 
 11  package_accomodation   18506 non-null  object 
 12  package_food           18506 non-null  object 
 13  package_transport_tz   18506 non-null  object 
 14  package_sightseeing    18506 non-null  object 
 15  pa

In [22]:
cat_features

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        19, 21, 22, 23], dtype=int64),)

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42) 

In [8]:
model=CatBoostClassifier(n_estimators=600,
                        max_depth=8,
                        learning_rate=0.1,
                        use_best_model=True,
                        eval_metric='Accuracy',
                        )

model.fit(X_train, y_train, eval_set=(X_test,y_test))

0:	learn: 0.5206701	test: 0.5108050	best: 0.5108050 (0)	total: 58.2ms	remaining: 34.9s
1:	learn: 0.5294515	test: 0.5140465	best: 0.5140465 (1)	total: 107ms	remaining: 31.9s
2:	learn: 0.5335720	test: 0.5172880	best: 0.5172880 (2)	total: 157ms	remaining: 31.2s
3:	learn: 0.5414753	test: 0.5205294	best: 0.5205294 (3)	total: 210ms	remaining: 31.3s
4:	learn: 0.5423534	test: 0.5205294	best: 0.5205294 (3)	total: 263ms	remaining: 31.2s
5:	learn: 0.5441097	test: 0.5210697	best: 0.5210697 (5)	total: 326ms	remaining: 32.3s
6:	learn: 0.5447852	test: 0.5240411	best: 0.5240411 (6)	total: 388ms	remaining: 32.8s
7:	learn: 0.5456633	test: 0.5205294	best: 0.5240411 (6)	total: 449ms	remaining: 33.2s
8:	learn: 0.5474872	test: 0.5259319	best: 0.5259319 (8)	total: 545ms	remaining: 35.8s
9:	learn: 0.5505269	test: 0.5256618	best: 0.5259319 (8)	total: 601ms	remaining: 35.5s
10:	learn: 0.5549176	test: 0.5324149	best: 0.5324149 (10)	total: 652ms	remaining: 34.9s
11:	learn: 0.5554580	test: 0.5305240	best: 0.532414

In [9]:
y_pred=model.predict(X_test)

In [10]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.35      0.52      0.42       518
           1       0.74      0.60      0.67      1181
           2       0.03      0.38      0.06         8
           3       0.03      0.22      0.05        41
           4       0.63      0.58      0.60       525
           5       0.73      0.56      0.64      1429

    accuracy                           0.57      3702
   macro avg       0.42      0.48      0.41      3702
weighted avg       0.66      0.57      0.60      3702

